# building a stock program

- STEP 1
    - monitor 3 or 4 diverse stocks (yfinance? finnhub?)
    - PCAR, TSLA, AMZN, CRUDE OIL
    - read in stock data for these
- STEP 2
    - pass their own data as endog and the others as exog into a walking sarima
- STEP 2.5
    - feature engeneering. What else can I add as exog to increase accuracy? Unemploymeent? Inflation? etc..
    - How should I incorpoporate things other than Close
- STEP 3
    - set up an account with a small amount of money and let this invest it freely
    - maybe in my schwab? maybe on investopedia?


- Thoughts
    - switch to day change
    - switch from forecasting to just classification sell/hold
    - look into stats forecast package

In [13]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import statsmodels.api as sm
import pmdarima as pm
import warnings
from sklearn.metrics import mean_squared_error
import math
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [2]:
test = yf.Ticker('amzn').history(period='7day')
test

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-08-12,142.050003,143.570007,140.119995,143.550003,47582800,0,0


In [3]:
pcar = yf.Ticker('PCAR').history(period='max')[['Close']]#,'High','Low','Volume']]
amzn = yf.Ticker('AMZN').history(period='max')[['Close']]#,'High','Low','Volume']]
tsla = yf.Ticker('TSLA').history(period='max')[['Close']]#,'High','Low','Volume']]
oil = yf.Ticker('CL').history(period='max')[['Close']]#,'High','Low','Volume']]
comp = yf.Ticker('comp').history(period='max')[['Close']]#,'High','Low','Volume']]
aapl = yf.Ticker('aapl').history(period='max')[['Close']]#,'High','Low','Volume']]


In [4]:
stocks = [pcar,amzn,tsla,oil,comp,aapl]
stock_name = ['pcar','amzn','tsla','oil','comp','aapl']


In [5]:
combo = pd.concat(stocks,axis=1)
combo.columns = stock_name
combo = combo.dropna()
combo.index = pd.to_datetime(combo.index)



In [273]:

combo2 = combo.copy()
lis2 = lis.copy()
bis = lis2.copy()
bis.remove('tsla')
print(bis)
combo2[bis]



['pcar', 'amzn', 'oil', 'comp', 'aapl']


,pcar,amzn,oil,comp,aapl
Date,,,,,
2021-04-01,88.747688,158.050003,75.896126,20.150000,121.924522
2021-04-05,90.608856,161.336502,76.843018,18.590000,124.799156
2021-04-06,89.538452,161.190994,77.161865,18.700001,125.106445
2021-04-07,89.172005,163.969498,76.929977,17.200001,126.781677
2021-04-08,89.528801,164.964996,76.949295,16.809999,129.220169
...,...,...,...,...,...
2022-08-08,91.589996,139.410004,80.809998,4.600000,164.869995
2022-08-09,91.370003,137.830002,80.059998,4.400000,164.919998
2022-08-10,93.250000,142.690002,80.339996,4.740000,169.240005


In [355]:
# look into endog params within model

def walking_sarimax(data,tt,params):
    walking_preds_list=[]
    j_preds=[]
    acts=[]
    lis = [x for x in combo.columns]
    order_ = params['order']
    seasonal_order_ = params['seasonal_order']
    trend_ = params['trend']
    for stock in lis:
        size = int(len(data)*tt)
        train, test = data[0:size],data[size:len(data)]
        test = np.append(test,np.nan)

        train_endog = train[[stock]]
        test_endog = test[[stock]]
        test_endog_ = [x for x in test_endog.values]
        history_endog = [x for x in train_endog.values]

        exs = lis.copy()
        exs.remove(stock)
        train_exog = train[exs]
        test_exog = test[exs]
        test_exog_ = [x for x in test_exog.values]
        history_exog = [x for x in train_exog.values]

        # needs to be yesterdays info from these stocks. Remove first one, add nan to end (nan added above)

        preds = [np.nan]*len(train)
        for t in range(len(test)):
            model = sm.tsa.statespace.SARIMAX(endog = history_endog,exog = history_exog,\
                order=order_,seasonal_order=seasonal_order_,trend=trend_,\
                    time_varying_regressionbool=True,mle_regression=False,initialization='approximate_diffuse' )

            model_fit = model.fit(disp=False)
            output = model_fit.forecast(exog = test_exog_[t])
            history_endog.append(test_endog_[t])
            history_exog.append(test_exog_[t])
            preds.append(output[0])
            
            if math.isnan(test_endog_[t]) == False:
                j_preds.append(output[0])
                acts.append(test_endog_[t])
        dates = np.append(np.array(data.index.date),(data.index.max()+pd.DateOffset(days=1))) # add tomro
        walking_preds = pd.DataFrame(zip(dates,data[stock],preds),columns=['Date',f'{stock}_Actual',f'{stock}_Pred'])
        walking_preds_list.append(walking_preds)
    walking_preds_all = pd.concat(walking_preds_list,axis=1).T.drop_duplicates().T
    return walking_preds_all,j_preds,acts

In [14]:
def walking_sarima(data,tt,params):
    walking_preds_list=[]
    j_preds=[]
    acts=[]
    for stock in combo.columns:
        X=data[stock].to_numpy()

        size = int(len(X)*tt)
        train, test = X[0:size],X[size:len(X)]
        #add an np.nan onto the end of train to represent the unknown tomorow
        test = np.append(test,np.nan)
                
        history = [x for x in train]

        preds = [np.nan]*len(train)

        order_ = params['order']
        seasonal_order_ = params['seasonal_order']
        trend_ = params['trend']
        for t in range(len(test)):
            model = sm.tsa.statespace.SARIMAX(history,order=order_,seasonal_order=seasonal_order_,trend=trend_)

            model_fit = model.fit(disp=False)
            output = model_fit.forecast()

            history.append(test[t])
            preds.append(output[0])
            
            if math.isnan(test[t]) == False:
                j_preds.append(output[0])
                acts.append(test[t])

        X = np.append(X,np.nan)
        dates = np.append(np.array(data.index.date),(data.index.max()+pd.DateOffset(days=1))) # add tomro
        walking_preds = pd.DataFrame(zip(dates,X,preds),columns=['Date',f'{stock}_Actual',f'{stock}_Pred'])
        walking_preds_list.append(walking_preds)
    walking_preds_all = pd.concat(walking_preds_list,axis=1).T.drop_duplicates().T
    return walking_preds_all,j_preds,acts

auto = pm.auto_arima(combo.spx, X=combo.drop('spx',axis=1).to_numpy(), start_p=2, d=None, start_q=2, max_p=15, max_d=2, max_q=7, start_P=1, D=None, 
start_Q=1, max_P=5, max_D=1, max_Q=2, max_order=5, m=1, seasonal=True, stationary=False, 
information_criterion='aic', alpha=0.05, test='kpss', seasonal_test='ocsb', stepwise=True, 
n_jobs=1, start_params=None, trend=None, method='lbfgs', maxiter=50, offset_test_args=None, 
seasonal_test_args=None, suppress_warnings=True, error_action='trace', trace=False, random=False, 
random_state=None, n_fits=10, return_valid_fits=False, out_of_sample_size=0, scoring='mse', scoring_args=None, 
with_intercept='auto', sarimax_kwargs=None)

In [8]:
params = {'order':[5,0,0],
          'seasonal_order':[0,0,0,5],
          'trend':'ct'}

In [356]:
outX,pX,aX= walking_sarimax(combo.tail(100),0.75,params)

print(f'sarimax: {mean_squared_error(pX,aX)}') #391

out,p,a = walking_sarima(combo.tail(100),0.75,params)

print(f'sarima: {mean_squared_error(p,a)}') #127.87




sarimax: 115.08360351269282
sarima: 208.53056817166373


In [357]:
px.line(outX.set_index('Date'))

# find some way to test
- start with X dollars
- if alg predicts price goes up, hold
- if alg predicts price goes down, sell

In [308]:
def decide(output,stock):
    # if next pred higher than current actual, hold, else, sell
    output = output[['Date',f'{stock}_Actual',f'{stock}_Pred']].dropna()
    todo = []
    act = f'{stock}_Actual'
    pre = f'{stock}_Pred'
    for i in range(len(output)-1):
        if output.iloc[i][act] <= output.iloc[i+1][pre]: # if tomorow pred is bigger than todays actual
            todo.append('HOLD')
        else:
            todo.append('SELL')
    todo.append('?')
    output['Todo'] = todo
    return output

In [347]:
compare_list=[]
for stock in stock_name:
    x = 100
    x2 = x
    x3 = x
    plann=[]
    hold=[]
    perfect=[]
    plan = decide(outX,stock)

    for i in range(len(plan)-1):
        row = plan.iloc[i]
        next_row = plan.iloc[i+1]
        
        change = ((next_row[1]-row[1])/row[1])
        # using system
        if row.Todo =='HOLD':
            x = x + (x*change)
        # just holding
        x2 = x2+(x2*change)
        # if it was perfect
        if change >0:
            x3 = x3+(x3*change)
        
        plann.append(x)
        hold.append(x2)
        perfect.append(x3)
    
    print(f'{stock}:')
    print(f'    -Perfect: {x3}$')
    print(f'    -Plan: {x}$')
    print(f'    -Hold: {x2}$')
    plan_frame = pd.DataFrame(zip(plan.Date,plann,['plan']*len(plann),[stock]*len(plann)),columns = ['date','val','type','stock'])
    hold_frame = pd.DataFrame(zip(plan.Date,hold,['hold']*len(plann),[stock]*len(plann)),columns = ['date','val','type','stock'])
    perfect_frame = pd.DataFrame(zip(plan.Date,perfect,['perfect']*len(plann),[stock]*len(plann)),columns = ['date','val','type','stock'])
    
    compare= pd.concat([plan_frame,hold_frame,perfect_frame])
    compare_list.append(compare)
all_compare = pd.concat(compare_list)



pcar:
    -Perfect: 189.1616494255718$
    -Plan: 144.24864179368936$
    -Hold: 113.3111132959963$
amzn:
    -Perfect: 316.72246583069096$
    -Plan: 256.71526166803756$
    -Hold: 92.93638678051343$
tsla:
    -Perfect: 401.037146994102$
    -Plan: 337.22253997395563$
    -Hold: 87.77170283736795$
oil:
    -Perfect: 157.0511193291423$
    -Plan: 118.96783187574157$
    -Hold: 101.25040817579243$
comp:
    -Perfect: 677.3840394022559$
    -Plan: 211.64954532644947$
    -Hold: 71.64633918081557$
aapl:
    -Perfect: 225.94260646877436$
    -Plan: 191.50943548928942$
    -Hold: 101.47115343784343$


In [343]:
all_compare['indicator'] = all_compare['type']+' '+all_compare['stock']

In [344]:
px.line(x=all_compare.date,y=all_compare.val,color=all_compare.indicator)